In [1]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
def get_expiry(f_today):
    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [5]:
bnf_pandas = pd.read_csv("../data/bnf.csv")

In [6]:
# symbol = 'midcp'
# symbol = 'nifty'
# symbol = 'fnf'
symbol = 'bnf'
# symbol = 'sensex'
# symbol = 'bankex'

if symbol == 'bnf' or symbol == 'bankex':
    LEVERAGE_ = 5
    LOT_SIZE_ = 15
    SLIPPAGE_ = 0.0001
elif symbol == 'nifty' or symbol == 'fnf':
    LEVERAGE_ = 5
    LOT_SIZE_ = 25
    SLIPPAGE_ = 0.0002
elif symbol == 'midcp':
    LEVERAGE_ = 12
    LOT_SIZE_ = 50
    SLIPPAGE_ = 0.0005
elif symbol == 'sensex':
    LEVERAGE_ = 8
    LOT_SIZE_ = 10
    SLIPPAGE_ = 0.0001

PORTFOLIO = 1_00_00_000
print(LEVERAGE_)

5


In [7]:
bnf_pandas.head()

,datetime,open,high,low,close,volume
0,2017-01-02 09:15:00,18242.3000,18248.2000,18175.9000,18181.2000,0
1,2017-01-02 09:16:00,18181.8500,18194.7000,18179.9500,18184.4500,0
2,2017-01-02 09:17:00,18184.9500,18189.2500,18133.8000,18133.8000,0
3,2017-01-02 09:18:00,18135.1000,18141.5500,18118.5500,18138.9500,0
4,2017-01-02 09:19:00,18138.9500,18142.5500,18120.4500,18124.3000,0


In [8]:
# If Stocks Data ...
bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["datetime"])
bnf_pandas["datetime"] = bnf_pandas["datetime"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["datetime"].dt.year >= 2017]
# bnf_pandas.drop(columns=["datetime"], inplace=True)
# bnf_pandas

In [9]:
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# bnf

<class 'polars.dataframe.frame.DataFrame'>


In [10]:
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")
bnf = bnf.with_columns(pl.col("index").alias("datetime"))
bnf.tail()
bnf_pandas = bnf.to_pandas()

In [11]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )


In [12]:
def macd_logic_from_df(df, fast_ma, slow_ma):
    
    df['close'] = pd.to_numeric(df['close'], errors='coerce')
    close_prices = df['close']
    
    fast_ema = close_prices.ewm(span=fast_ma, adjust=False).mean()
    slow_ema = close_prices.ewm(span=slow_ma, adjust=False).mean()

    macd = slow_ema - fast_ema
    # prev_macd = macd.shift(1)
    
    df['macd'] = macd
    # df['prev_macd'] = prev_macd
    
    return df

In [13]:
# # Positional

# def execute(df, sl_pct):

#     trade_book = []
#     in_trade = False
#     signal_entry_price = 100000
#     signal_initial_sl = 0
#     already_signal_exists = False
#     is_trailing_active = False
#     remark = ""

#     for i in range(1, len(df)):
#         points = 0
#         current_candle_open = df.iloc[i]["open"]
#         current_candle_high = df.iloc[i]["high"]
#         current_candle_low = df.iloc[i]["low"]
#         current_candle_close = df.iloc[i]["close"]
#         current_macd = df.iloc[i]['macd']
#         current_ma = df.iloc[i]['ma']

#         if not in_trade:
#             if df.iloc[i-1]['macd'] > 0:
#                 # Previous Candle has a signal
#                 signal_entry_price = df.iloc[i-1]['high']
#                 signal_initial_sl = signal_entry_price * (1 - sl_pct)
#                 signal_generation_time = df.iloc[i-1]['datetime']

#                 if current_candle_high >= signal_entry_price:
#                     if current_candle_open >= signal_entry_price:
#                         if current_candle_low <= signal_entry_price:
#                             # Entry Triggered
#                             in_trade = True
#                             entry_time = df.iloc[i]['datetime']
#                             entry_price = signal_entry_price
#                             points = 0
#                         else:
#                             # Trade Skipped, Gap Open Outside Entry
#                             continue
#                     else:
#                         # Entry Triggered
#                         in_trade = True
#                         entry_time = df.iloc[i]['datetime']
#                         entry_price = signal_entry_price
#                         points = 0
#                 else:
#                     # Check Next Iteration for Better Candle Scenario
#                     continue

#         if in_trade:
#             trade_entry_price = signal_entry_price
#             trade_initial_sl = signal_initial_sl
#             trade_final_sl = signal_initial_sl
            
#             if (
#                 not is_trailing_active
#                 and current_candle_low > current_ma
#             ):
#                 is_trailing_active = True

#             if not is_trailing_active:
#                 if current_candle_open <= signal_initial_sl:
#                     if (
#                         df.iloc[i]["datetime"].date() == entry_time.date()
#                         and df.iloc[i]["datetime"].time() == entry_time.time()
#                     ):
#                         # Check if this Gap Open below SL candle is the Entry Candle
#                         if current_candle_close <= trade_initial_sl:
#                             in_trade = False
#                             points = trade_initial_sl - trade_entry_price
#                             exit_price = trade_initial_sl
#                             exit_time = df.iloc[i]["datetime"]
#                             remark = "Initial SL hit"

#                     else:
#                         # Gap Open Outside ISL
#                         in_trade = False
#                         points = current_candle_open - trade_entry_price
#                         exit_price = current_candle_open
#                         exit_time = df.iloc[i]["datetime"]
#                         remark = "Gap Open Outside ISL"

#                 elif current_candle_low <= trade_initial_sl:
#                     # Initial SL Hit
#                     in_trade = False
#                     points = trade_initial_sl - trade_entry_price
#                     exit_price = trade_initial_sl
#                     exit_time = df.iloc[i]["datetime"]
#                     remark = "Initial SL Hit"

#                 elif df.iloc[i]['macd'] < 0:
#                     # MACD Reversal
#                     in_trade = False
#                     points = current_candle_close - trade_entry_price
#                     exit_price = current_candle_close
#                     exit_time = df.iloc[i]["datetime"]
#                     remark = "MACD Reversal"

#             else:
#                 trade_final_sl = max(trade_initial_sl, current_ma)

#                 if current_candle_open <= trade_initial_sl:
#                     if (
#                         df.iloc[i]["datetime"].date() == entry_time.date()
#                         and df.iloc[i]["datetime"].time() == entry_time.time()
#                     ):
#                         if current_candle_close <= trade_initial_sl:
#                             in_trade = False
#                             points = trade_initial_sl - trade_entry_price
#                             exit_price = trade_initial_sl
#                             exit_time = df.iloc[i]["datetime"]
#                             remark = "Initial SL hit"

#                     else:
#                         # Gap Open Outside ISL
#                         in_trade = False
#                         points = current_candle_open - trade_entry_price
#                         exit_price = current_candle_open
#                         exit_time = df.iloc[i]["datetime"]
#                         remark = "Gap Open Outside ISL"

#                 elif current_candle_low <= trade_initial_sl:
#                     # Despite Trailing, Initial SL hit
#                     in_trade = False
#                     points = trade_initial_sl - trade_entry_price
#                     exit_price = trade_initial_sl
#                     exit_time = df.iloc[i]["datetime"]
#                     remark = "Initial SL hit"

#                 elif current_candle_close <= trade_final_sl:
#                     # Price Closed below TSL
#                     in_trade = False
#                     points = current_candle_close - trade_entry_price
#                     exit_price = current_candle_close
#                     exit_time = df.iloc[i]["datetime"]
#                     is_trailing_active = False
#                     remark = "TSL Hit"

#                 elif df.iloc[i]['macd'] < 0:
#                     # MACD Reversal
#                     in_trade = False
#                     points = current_candle_close - trade_entry_price
#                     exit_price = current_candle_close
#                     exit_time = df.iloc[i]["datetime"]
#                     remark = "MACD Reversal"

#             if points:
#                 qty = int(round(PORTFOLIO * LEVERAGE_ / entry_price / LOT_SIZE_)) * LOT_SIZE_
#                 slippage = SLIPPAGE_ * (entry_price + exit_price)
#                 # slippage = 10
#                 final_points = points - slippage
#                 # final_points = points
#                 trade = {
#                     "Signal Generated At": signal_generation_time,
#                     "Trade Type": "LONG",
#                     "Entry Time": entry_time,
#                     "Entry Price": entry_price,
#                     "Initial SL": trade_initial_sl,
#                     "Final SL": trade_final_sl,
#                     "Exit Time": exit_time,
#                     "Exit Price": exit_price,
#                     "Points Captured": points,
#                     "After Costs": final_points,
#                     "PnL": final_points * qty,
#                     "Remarks": remark,
#                     "Qty": qty,
#                     "Leverage": LEVERAGE_,
#                     "ROI%": (final_points * qty / PORTFOLIO) * 100,
#                     "Trade Year": entry_time.year,
#                     "Trade Month": entry_time.month,
#                 }
#                 # print(trade)
#                 trade_book.append(trade)
#                 points = 0
#                 in_trade = False
#                 already_signal_exists = False
#                 remark = ""
#                 is_trailing_active = False

#     trade_book_df = pd.DataFrame(trade_book)
#     return trade_book_df

In [16]:
# Positional

def execute(df, sl_pct):

    trade_book = []
    in_trade = False
    signal_entry_price = 100000
    signal_initial_sl = 0
    already_signal_exists = False
    is_trailing_active = False
    remark = ""

    for i in range(1, len(df)):
        points = 0
        current_candle_open = df.iloc[i]["open"]
        current_candle_high = df.iloc[i]["high"]
        current_candle_low = df.iloc[i]["low"]
        current_candle_close = df.iloc[i]["close"]
        current_macd = df.iloc[i]['macd']
        current_ma = df.iloc[i]['ma']

        if not in_trade:
            if (df.iloc[i-2]['macd'] < df.iloc[i-1]['macd']) and (df.iloc[i-1]['macd'] < 0):
                # Previous Candle has a signal
                signal_entry_price = df.iloc[i-1]['high']
                signal_initial_sl = signal_entry_price * (1 - sl_pct)
                signal_generation_time = df.iloc[i-1]['datetime']

                if current_candle_high >= signal_entry_price:
                    if current_candle_open >= signal_entry_price:
                        if current_candle_low <= signal_entry_price:
                            # Entry Triggered
                            in_trade = True
                            entry_time = df.iloc[i]['datetime']
                            entry_price = signal_entry_price
                            points = 0
                        else:
                            # Trade Skipped, Gap Open Outside Entry
                            continue
                    else:
                        # Entry Triggered
                        in_trade = True
                        entry_time = df.iloc[i]['datetime']
                        entry_price = signal_entry_price
                        points = 0
                else:
                    # Check Next Iteration for Better Candle Scenario
                    continue

        if in_trade:
            trade_entry_price = signal_entry_price
            trade_initial_sl = signal_initial_sl
            trade_final_sl = signal_initial_sl
            
            if (
                not is_trailing_active
                and current_candle_low > current_ma
            ):
                is_trailing_active = True

            if not is_trailing_active:
                if current_candle_open <= signal_initial_sl:
                    if (
                        df.iloc[i]["datetime"].date() == entry_time.date()
                        and df.iloc[i]["datetime"].time() == entry_time.time()
                    ):
                        # Check if this Gap Open below SL candle is the Entry Candle
                        if current_candle_close <= trade_initial_sl:
                            in_trade = False
                            points = trade_initial_sl - trade_entry_price
                            exit_price = trade_initial_sl
                            exit_time = df.iloc[i]["datetime"]
                            remark = "Initial SL hit"

                    else:
                        # Gap Open Outside ISL
                        in_trade = False
                        points = current_candle_open - trade_entry_price
                        exit_price = current_candle_open
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Gap Open Outside ISL"

                elif current_candle_low <= trade_initial_sl:
                    # Initial SL Hit
                    in_trade = False
                    points = trade_initial_sl - trade_entry_price
                    exit_price = trade_initial_sl
                    exit_time = df.iloc[i]["datetime"]
                    remark = "Initial SL Hit"

                elif (df.iloc[i]['macd'] < df.iloc[i-1]['macd']) and (df.iloc[i-1]['macd'] > 0):
                    # MACD Reversal
                    in_trade = False
                    points = current_candle_close - trade_entry_price
                    exit_price = current_candle_close
                    exit_time = df.iloc[i]["datetime"]
                    remark = "MACD Reversal"

            else:
                trade_final_sl = max(trade_initial_sl, current_ma)

                if current_candle_open <= trade_initial_sl:
                    if (
                        df.iloc[i]["datetime"].date() == entry_time.date()
                        and df.iloc[i]["datetime"].time() == entry_time.time()
                    ):
                        if current_candle_close <= trade_initial_sl:
                            in_trade = False
                            points = trade_initial_sl - trade_entry_price
                            exit_price = trade_initial_sl
                            exit_time = df.iloc[i]["datetime"]
                            remark = "Initial SL hit"

                    else:
                        # Gap Open Outside ISL
                        in_trade = False
                        points = current_candle_open - trade_entry_price
                        exit_price = current_candle_open
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Gap Open Outside ISL"

                elif current_candle_low <= trade_initial_sl:
                    # Despite Trailing, Initial SL hit
                    in_trade = False
                    points = trade_initial_sl - trade_entry_price
                    exit_price = trade_initial_sl
                    exit_time = df.iloc[i]["datetime"]
                    remark = "Initial SL hit"

                elif current_candle_close <= trade_final_sl:
                    # Price Closed below TSL
                    in_trade = False
                    points = current_candle_close - trade_entry_price
                    exit_price = current_candle_close
                    exit_time = df.iloc[i]["datetime"]
                    is_trailing_active = False
                    remark = "TSL Hit"

                elif (df.iloc[i]['macd'] < df.iloc[i-1]['macd']) and (df.iloc[i-1]['macd'] > 0):
                    # MACD Reversal
                    in_trade = False
                    points = current_candle_close - trade_entry_price
                    exit_price = current_candle_close
                    exit_time = df.iloc[i]["datetime"]
                    remark = "MACD Reversal"

            if points:
                qty = int(round(PORTFOLIO * LEVERAGE_ / entry_price / LOT_SIZE_)) * LOT_SIZE_
                slippage = SLIPPAGE_ * (entry_price + exit_price)
                # slippage = 10
                final_points = points - slippage
                # final_points = points
                trade = {
                    "Signal Generated At": signal_generation_time,
                    "Trade Type": "LONG",
                    "Entry Time": entry_time,
                    "Entry Price": entry_price,
                    "Initial SL": trade_initial_sl,
                    "Final SL": trade_final_sl,
                    "Exit Time": exit_time,
                    "Exit Price": exit_price,
                    "Points Captured": points,
                    "After Costs": final_points,
                    "PnL": final_points * qty,
                    "Remarks": remark,
                    "Qty": qty,
                    "Leverage": LEVERAGE_,
                    "ROI%": (final_points * qty / PORTFOLIO) * 100,
                    "Trade Year": entry_time.year,
                    "Trade Month": entry_time.month,
                }
                # print(trade)
                trade_book.append(trade)
                points = 0
                in_trade = False
                already_signal_exists = False
                remark = ""
                is_trailing_active = False

    trade_book_df = pd.DataFrame(trade_book)
    return trade_book_df

In [17]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio : stats_df8}

In [36]:
stats_dictionary = {}

slow_ma = 20
fast_ma = 5
trailing_ma = 40
sl_pct = 0.0075
df1 = resample(pl.DataFrame(bnf_pandas), '60m', '15m')
df2 = df1.to_pandas()
df = macd_logic_from_df(df2, slow_ma, fast_ma)
# df = macd_logic_from_df(bnf_pandas, slow_ma, fast_ma)
df['ma'] = df['close'].rolling(window=trailing_ma).mean()
tb = execute(df, sl_pct)
variation = f'MA1 : {slow_ma}, MA2 : {fast_ma}, SL : {sl_pct * 100}%'

In [37]:
stats = generate_stats(tb, variation)

In [38]:
for overall_roi_dd_ratio, stats_df in stats.items():
    if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > -10:
        print(stats_df.to_string())
        stats_dictionary[overall_roi_dd_ratio] = stats_df

        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio                      Variation
2017     112.7018           48  58.3333                6.3976             -3.3216     -12.5213       9.0008  MA1 : 20, MA2 : 5, SL : 0.75%
2018       6.6708           71  40.8451                6.1404             -4.0810     -70.6796       0.0944  MA1 : 20, MA2 : 5, SL : 0.75%
2019     121.4645           66  46.9697                7.9255             -3.5493     -23.5897       5.1491  MA1 : 20, MA2 : 5, SL : 0.75%
2020      31.1435          108  26.8519               15.2027             -5.1865    -188.6642       0.1651  MA1 : 20, MA2 : 5, SL : 0.75%
2021      85.8147           82  39.0244                8.5393             -3.7489     -51.9776       1.6510  MA1 : 20, MA2 : 5, SL : 0.75%
2022      16.0633           83  34.9398                7.6966             -3.8359     -76.5039       0.2100  MA1 : 20, MA2 : 5, SL : 0.75%
2023      11.6715          

In [39]:
tb.tail()

,Signal Generated At,Trade Type,Entry Time,Entry Price,Initial SL,Final SL,Exit Time,Exit Price,Points Captured,After Costs,PnL,Remarks,Qty,Leverage,ROI%,Trade Year,Trade Month
554,2024-05-22 13:15:00,LONG,2024-05-22 14:15:00,47803.5500,47445.0234,47445.0234,2024-05-22 14:15:00,47445.0234,-358.5266,-368.0515,-386454.0565,Initial SL Hit,1050,5,-3.8645,2024,5
555,2024-05-22 14:15:00,LONG,2024-05-22 15:15:00,47851.5000,47492.6138,48052.7562,2024-05-24 09:15:00,48679.1500,827.6500,817.9969,858896.7818,MACD Reversal,1050,5,8.5890,2024,5
556,2024-05-29 15:15:00,LONG,2024-05-30 09:15:00,48624.9000,48260.2133,49266.5712,2024-06-04 09:15:00,49525.6500,900.7500,890.9349,922117.6681,MACD Reversal,1035,5,9.2212,2024,5
557,2024-06-05 10:15:00,LONG,2024-06-05 11:15:00,47865.9000,47506.9058,47506.9058,2024-06-05 11:15:00,47506.9058,-358.9942,-368.5315,-386958.1071,Initial SL Hit,1050,5,-3.8696,2024,6
558,2024-06-05 11:15:00,LONG,2024-06-05 12:15:00,48000.6000,47640.5955,48852.9675,2024-06-06 11:15:00,49231.1500,1230.5500,1220.8268,1263555.7639,MACD Reversal,1035,5,12.6356,2024,6


In [ ]:
# tb.to_csv('MACD_midcp.csv')

In [ ]:
# This is BNF

In [ ]:
stats_dictionary = {}
# i = slow_ma , j = fast_ma , k = trailing_ma
tfs = ['15m', '30m']
sl_pct = [0.015, 0.0125, 0.01, 0.008, 0.0066, 0.005]
for i in range(5, 51, 5):
    for j in range(5, 21, 5):
        for k in range(10, 81, 5):
            for sl in sl_pct:
                for tf in tfs:
                    if tf == '30m':
                        ofs = '15m'
                    else:
                        ofs = '0m'
                        
                    if i > j:
                        variation = f'MA1 : {i}, MA2 : {j}, SL : {sl * 100}% , TMA : {k}, TF : {tf}'
                        print(variation)
                        df1 = resample(pl.DataFrame(bnf_pandas), tf, ofs)
                        df2 = df1.to_pandas()
                        df = macd_logic_from_df(df2, i, j)
                        df['ma'] = df['close'].rolling(window=k).mean()
                        # print(df.tail().to_string())
                        new_tb = execute(df, sl)
                        if len(new_tb) > 1:
                            new_tb["DD%"] = new_tb["ROI%"].cumsum() - new_tb["ROI%"].cumsum().cummax()
                            tradebook_buy_side = new_tb
                            stats = generate_stats(tradebook_buy_side, variation)
                            for overall_roi_dd_ratio, stats_df in stats.items():
                                if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 12:
                                    print(stats_df.to_string())
                                    stats_dictionary[overall_roi_dd_ratio] = stats_df

MA1 : 10, MA2 : 5, SL : 1.5% , TMA : 10, TF : 15m
MA1 : 10, MA2 : 5, SL : 1.5% , TMA : 10, TF : 30m
MA1 : 10, MA2 : 5, SL : 1.25% , TMA : 10, TF : 15m
MA1 : 10, MA2 : 5, SL : 1.25% , TMA : 10, TF : 30m
MA1 : 10, MA2 : 5, SL : 1.0% , TMA : 10, TF : 15m
MA1 : 10, MA2 : 5, SL : 1.0% , TMA : 10, TF : 30m
MA1 : 10, MA2 : 5, SL : 0.8% , TMA : 10, TF : 15m
MA1 : 10, MA2 : 5, SL : 0.8% , TMA : 10, TF : 30m
MA1 : 10, MA2 : 5, SL : 0.66% , TMA : 10, TF : 15m
MA1 : 10, MA2 : 5, SL : 0.66% , TMA : 10, TF : 30m
MA1 : 10, MA2 : 5, SL : 0.5% , TMA : 10, TF : 15m
MA1 : 10, MA2 : 5, SL : 0.5% , TMA : 10, TF : 30m
MA1 : 10, MA2 : 5, SL : 1.5% , TMA : 15, TF : 15m
MA1 : 10, MA2 : 5, SL : 1.5% , TMA : 15, TF : 30m
MA1 : 10, MA2 : 5, SL : 1.25% , TMA : 15, TF : 15m
MA1 : 10, MA2 : 5, SL : 1.25% , TMA : 15, TF : 30m
MA1 : 10, MA2 : 5, SL : 1.0% , TMA : 15, TF : 15m
MA1 : 10, MA2 : 5, SL : 1.0% , TMA : 15, TF : 30m
MA1 : 10, MA2 : 5, SL : 0.8% , TMA : 15, TF : 15m
MA1 : 10, MA2 : 5, SL : 0.8% , TMA : 15, TF 

In [ ]:
sorted_stats = {k: v for k, v in sorted(stats_dictionary.items(), key=lambda item: item[0], reverse=True)}
sorted_stats

# BNF Automation Below

In [168]:
symbol = 'bnf'
# symbol = 'nifty'
# symbol = 'fnf'
# symbol = 'midcp'

if symbol == 'bnf':
    LEVERAGE_ = 5
    LOT_SIZE_ = 15
    SLIPPAGE_ = 0.0001
elif symbol == 'nifty' or 'fnf':
    LEVERAGE_ = 7
    LOT_SIZE_ = 25
    SLIPPAGE_ = 0.0002
elif symbol == 'midcp':
    LEVERAGE_ = 10
    LOT_SIZE_ = 50
    SLIPPAGE_ = 0.0005

PORTFOLIO = 1_00_00_000

In [169]:
bnf_pandas = pd.read_csv("../data/bnf_1hr_tv_new.csv")
# If Stocks Data ...
bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["time"])
bnf_pandas["datetime"] = bnf_pandas["datetime"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["datetime"].dt.year >= 2017]
# bnf_pandas.drop(columns=["datetime"], inplace=True)
# bnf_pandas
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# bnf
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")
bnf = bnf.with_columns(pl.col("index").alias("datetime"))
bnf.tail()
bnf_pandas = bnf.to_pandas()

<class 'polars.dataframe.frame.DataFrame'>


In [140]:
bnf_pandas.tail()

,time,open,high,low,close,index,datetime,macd,ma
13376,2024-09-27T11:15:00+05:30,54217.9000,54231.3000,54148.0500,54184.7000,2024-09-27 11:15:00,2024-09-27 11:15:00,144.7985,54168.4933
13377,2024-09-27T12:15:00+05:30,54184.4000,54188.1500,53945.6500,53974.6000,2024-09-27 12:15:00,2024-09-27 12:15:00,109.7174,54165.0367
13378,2024-09-27T13:15:00+05:30,53971.5500,53978.9000,53830.6000,53903.2500,2024-09-27 13:15:00,2024-09-27 13:15:00,74.2038,54156.2533
13379,2024-09-27T14:15:00+05:30,53895.5000,53945.1000,53763.2000,53833.0000,2024-09-27 14:15:00,2024-09-27 14:15:00,38.9756,54140.0800
13380,2024-09-27T15:15:00+05:30,53834.7500,53876.7000,53787.5000,53834.3000,2024-09-27 15:15:00,2024-09-27 15:15:00,11.9610,54121.7867


In [155]:
stats_dictionary = {}
# i = slow_ma , j = fast_ma , k = trailing_ma

sl_pct = [0.015, 0.0125, 0.01, 0.0075, 0.0066, 0.005]
for i in range(5, 51, 5):
    for j in range(5, 21, 5):
        for k in range(10, 81, 5):
            for sl in sl_pct:
                if i > j:
                    variation = f'MA1 : {i}, MA2 : {j}, SL : {sl * 100}% , TMA : {k}'
                    print(variation)
                    df = macd_logic_from_df(bnf_pandas, i, j)
                    df['ma'] = df['close'].rolling(window=k).mean()
                    new_tb = execute(df, sl)
                    # print(new_tb.tail())
                    if len(new_tb) > 1:
                        new_tb["DD%"] = new_tb["ROI%"].cumsum() - new_tb["ROI%"].cumsum().cummax()
                        tradebook_buy_side = new_tb
                        stats = generate_stats(tradebook_buy_side, variation)
                        for overall_roi_dd_ratio, stats_df in stats.items():
                            if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 8:
                                print(stats_df.to_string())
                                stats_dictionary[overall_roi_dd_ratio] = stats_df

MA1 : 10, MA2 : 5, SL : 1.5% , TMA : 10
MA1 : 10, MA2 : 5, SL : 1.25% , TMA : 10
MA1 : 10, MA2 : 5, SL : 1.0% , TMA : 10
MA1 : 10, MA2 : 5, SL : 0.75% , TMA : 10
MA1 : 10, MA2 : 5, SL : 0.66% , TMA : 10
MA1 : 10, MA2 : 5, SL : 0.5% , TMA : 10
MA1 : 10, MA2 : 5, SL : 1.5% , TMA : 15
MA1 : 10, MA2 : 5, SL : 1.25% , TMA : 15
MA1 : 10, MA2 : 5, SL : 1.0% , TMA : 15
MA1 : 10, MA2 : 5, SL : 0.75% , TMA : 15
MA1 : 10, MA2 : 5, SL : 0.66% , TMA : 15
MA1 : 10, MA2 : 5, SL : 0.5% , TMA : 15
MA1 : 10, MA2 : 5, SL : 1.5% , TMA : 20
MA1 : 10, MA2 : 5, SL : 1.25% , TMA : 20
MA1 : 10, MA2 : 5, SL : 1.0% , TMA : 20
MA1 : 10, MA2 : 5, SL : 0.75% , TMA : 20
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio                                 Variation
2022     152.5614           67  35.8209               16.7352             -5.7926     -54.7512       2.7864  MA1 : 10, MA2 : 5, SL : 0.75% , TMA : 20
2023     234.3277           59  38.9831             

# MIDCP Sorted Stats

In [156]:
sorted_stats = {k: v for k, v in sorted(stats_dictionary.items(), key=lambda item: item[0], reverse=True)}
sorted_stats

{10.338900077584734:         Total ROI Total Trades Win Rate Avg Profit% per Trade  \
 2022     168.9855           64  35.9375               18.2169   
 2023     239.6484           55  40.0000               17.5255   
 2024     150.4351           44  45.4545               15.1360   
 Overall  559.0690          163  39.8773               17.0349   
 
         Avg Loss% per Trade Max Drawdown ROI/DD Ratio  \
 2022                -6.0977     -54.0743       3.1251   
 2023                -4.4216     -39.3396       6.0918   
 2024                -6.3452     -30.0561       5.0052   
 Overall             -5.5939     -54.0743      10.3389   
 
                                         Variation  
 2022     MA1 : 10, MA2 : 5, SL : 0.66% , TMA : 80  
 2023     MA1 : 10, MA2 : 5, SL : 0.66% , TMA : 80  
 2024     MA1 : 10, MA2 : 5, SL : 0.66% , TMA : 80  
 Overall  MA1 : 10, MA2 : 5, SL : 0.66% , TMA : 80  ,
 10.264616877632214:         Total ROI Total Trades Win Rate Avg Profit% per Trade  \
 202